In [2]:
region_list = []
with open("data.csv") as file:
    i = iter(file)
    i.__next__()
    for line in i:
        if line.split("\t")[4] not in region_list:
            region_list.append(line.split("\t")[4])
        else: continue
region_list.sort()
region_list


['British Isles',
 'Central America',
 'Eastern Europe',
 'North America',
 'Northern Europe',
 'Scandinavia',
 'South America',
 'Southern Europe',
 'Western Europe']

In [3]:
import sqlite3
from sqlite3 import Error
def create_connection(db_file, delete_db=False):
    
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows

In [4]:
create_table_sql = """CREATE TABLE [Region] (
    [RegionID] Integer not null primary key,
    [Region] Text not null
    );
    """
conn= create_connection("hello.db", True)
create_table(conn, create_table_sql)

In [5]:
def insert_region(conn, values):
    
    sql = ''' INSERT INTO Region(Region)
            VALUES(?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid
    
# That the insert_region function is ready, we can start putting in values in the table as follows!
conn_norm = create_connection("hello.db")
with conn_norm:
    for region in region_list:
        insert_region(conn_norm, (region, ))

In [7]:
# Fetching all the degrees in a list
sql_statement = "SELECT Region, RegionID from Region"
regions_from_table = execute_sql_statement(sql_statement, conn)
regions_from_table

region_to_regionid_dictionary = {}

for i in range(len(regions_from_table)):
    region_to_regionid_dictionary[regions_from_table[i][0]] = regions_from_table[i][1]
    
region_to_regionid_dictionary

{'British Isles': 1,
 'Central America': 2,
 'Eastern Europe': 3,
 'North America': 4,
 'Northern Europe': 5,
 'Scandinavia': 6,
 'South America': 7,
 'Southern Europe': 8,
 'Western Europe': 9}